In [2]:
from transformers import AutoImageProcessor, PvtModel
import torch
!pip install datasets
from datasets import load_dataset

#Загрузка и предобработка изображения
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]
image_processor = AutoImageProcessor.from_pretrained("Zetatech/pvt-tiny-224")
inputs = image_processor(image, return_tensors="pt")

#Загрузка и запуск модели
model = PvtModel.from_pretrained("Zetatech/pvt-tiny-224")
outputs = model(**inputs)
print(outputs)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BaseModelOutput(last_hidden_state=tensor([[[-0.3086,  1.0402,  1.1816,  ..., -0.4371,  1.1505, -1.2348],
         [-0.2880,  0.5781,  0.6124,  ...,  2.5609,  0.9653,  0.2605],
         [ 0.1480,  0.6129, -0.0590,  ...,  2.5609,  0.7097, -0.3323],
         ...,
         [-0.1919,  0.5508, -0.9730,  ..., -0.1239, -1.3483,  0.1781],
         [ 0.0774,  0.7725, -1.8221,  ...,  0.0458, -1.3402, -0.5406],
         [-0.3391,  1.0366, -1.4478,  ...,  0.0894, -0.8115, -0.2258]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [ ]:
!pip install onnx onnxruntime
import onnx
import onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


In [ ]:
torch.onnx.export(model, tuple(inputs.values()), "pvt.onnx")

/usr/local/lib/python3.10/dist-packages/transformers/models/pvt/modeling_pvt.py:140: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/usr/local/lib/python3.10/dist-packages/transformers/models/pvt/modeling_pvt.py:131: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_patches == self.config.image_size * self.config.image_size:


In [ ]:
onnx_model = onnx.load("pvt.onnx")
ort_session = onnxruntime.InferenceSession("pvt.onnx")

In [ ]:
import numpy as np
#Изменение формы входных данных (ранее выдавало ошибку)
input_data = inputs['pixel_values'].numpy()
reshaped_input = np.reshape(input_data, (1, 3, 224, 224))

#Запуск модели
ort_inputs = {ort_session.get_inputs()[0].name: reshaped_input}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

[array([[[-0.30862734,  1.0402286 ,  1.1815618 , ..., -0.43709645,
          1.1505104 , -1.2347753 ],
        [-0.28796977,  0.57806975,  0.61235654, ...,  2.5609367 ,
          0.9653076 ,  0.26047134],
        [ 0.14797397,  0.6128762 , -0.05897992, ...,  2.5609117 ,
          0.7096809 , -0.3322651 ],
        ...,
        [-0.19194183,  0.55081606, -0.97296095, ..., -0.12386642,
         -1.3483355 ,  0.17808405],
        [ 0.07744904,  0.7724615 , -1.8221457 , ...,  0.04582934,
         -1.3402148 , -0.54059446],
        [-0.33906335,  1.036641  , -1.4477919 , ...,  0.08941542,
         -0.8114966 , -0.2257848 ]]], dtype=float32)]


In [ ]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("pvt.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.4 MB/s eta 0:00:00


In [ ]:
outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.3086,  1.0402,  1.1816,  ..., -0.4371,  1.1505, -1.2348],
         [-0.2880,  0.5781,  0.6124,  ...,  2.5609,  0.9653,  0.2605],
         [ 0.1480,  0.6129, -0.0590,  ...,  2.5609,  0.7097, -0.3323],
         ...,
         [-0.1919,  0.5508, -0.9730,  ..., -0.1239, -1.3483,  0.1781],
         [ 0.0774,  0.7725, -1.8221,  ...,  0.0458, -1.3402, -0.5406],
         [-0.3391,  1.0366, -1.4478,  ...,  0.0894, -0.8115, -0.2258]]]), hidden_states=None, attentions=None)
